Импорты

In [ ]:
import pandas as pd 
import json 
import requests 
from bs4 import BeautifulSoup 
from tqdm.auto import tqdm 

Считывание датасета

In [1]:
file_path = 'dataset.json' 
with open(file_path, 'r', encoding='utf-8') as file: 
    data = json.load(file) 
 
df = pd.json_normalize(data['data']) 
df = df.loc[df['description'] != 'фейк']

Функции для парсинга

In [2]:
def download_page(url): 
    try: 
        response = requests.get(url) 
        response.raise_for_status() 
        response.encoding = 'utf-8'
        return response.text 
    except requests.RequestException as e: 
        print(f"Error downloading the page: {e}") 
        return None 
 
def extract_text(soup, count): 
    return soup.find_all('article')[count].stripped_strings

Парсинг страниц

In [4]:
error_urls = []
infos = [] 
parent_url = ''
count = 0
texts = []
for uri in tqdm(df['url']):
    uri_parent = uri.split('?')[0]
    if parent_url == uri_parent:
        count+=1
    else:
        parent_url = uri_parent
        count=0
    html_content = download_page(uri) 
    if html_content: 
        soup = BeautifulSoup(html_content, 'html.parser') 
        try:
            text_content = extract_text(soup, count) 
        except:
            try:
                count = 0
                text_content = extract_text(soup, count) 
            except:
                error_urls.append(uri)
        infos.append(uri)
        texts.append(text_content)

df_texts = pd.DataFrame({'url': infos, 'text': texts})
df_errors = pd.DataFrame({'url': error_urls})

df_texts.to_csv('text.csv') # parsed urls
df_errors.to_csv('erros.csv') # unparsed urls for manual processing